In [250]:
# Load required libraries
import numpy as np
import pandas as pd
import joblib
import time
from IPython.core.display import clear_output

In [340]:
datafile = '/home/mosto/Documents/insight/kickstarter-project/web_robots_data_to_08-2020_processed.pkl'

In [341]:
features_table = '/home/mosto/Documents/insight/kickstarter-project/features_full_1.pkl'

Load the table containing already-parsed Web Robots data.

In [342]:
# Load table containing processed image features data
features = joblib.load(features_table)

In [343]:
# Load table containing Web Robots data
df = joblib.load(datafile)

The parsed data includes:

- `name` - project's name
- `category` - project's category as defined by Kickstarter
- `hyperlink` - project's web page URL
- `currency` - type of currency used for fundraising
- `pledged` - total amount of money pledged by backers over the course of the project
- `goal` - funding goal set by the creator
- `location` - creator's location information

Let's select the projects that only use U.S. dollars to ensure we're working with projects written in American English.

In [344]:
# Select projects described in American English
df_USD = df[df['currency'] == 'USD']

In [345]:
# Take a random sample of the Web Robots data using a seed value to ensure
# repeatability
seed = np.random.seed(74)
df_sample = df_USD.sample(50000)

Since the Web Robots data contains nearly 200,000 projects, we'll select a random sample for image scraping datasets.

In [346]:
df_sample.index.shape

(50000,)

In [347]:
df1_sample=df_sample.iloc[:10000]
df1_sample.shape

(10000, 17)

In [305]:
features.iloc[0]['image']

94899.0

In [348]:
# Label encode the sucess and faile states
cleanup_nums = {'state':{'successful':1, 'failed':0, 'live':2, 'canceled':0}, 'staff_pick':{False:0, True:1}}
df1_sample.replace(cleanup_nums, inplace=True)
df1_sample.head()

/home/mosto/miniconda3/envs/kickstarter/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,name,id,blurb,category,url,currency,pledged,goal,state,location,creator_id,launched_at,deadline,backers,staff_pick,creator_name,created_at
3990,Glow Girls start up fund,2896853,We are raising funds for Marketing purposes su...,DIY,https://www.kickstarter.com/projects/165542940...,USD,541,500,1,"Seattle, WA",1655429407,1488242943,1490831343,12,0.0,Claudia,1.488210e+09
124723,Studio for Artists,2738647,Unexpected health issues has taken it's toll o...,Painting,https://www.kickstarter.com/projects/117655044...,USD,50,5000,0,"Naples, FL",1176550444,1477960965,1480528140,1,0.0,Arthur Morehead,1.477769e+09
52182,Quickstarter: Mechanized Fabrications-What is ...,3736756,"A look into a world lost to progress, and the ...",Comic Books,https://www.kickstarter.com/projects/mechanize...,USD,317,300,1,"Alexandria, VA",385588855,1559061039,1560789039,11,0.0,Sebastian J,1.558452e+09
43468,Photography Business launch for Joseph in Nair...,3778504,Joseph Koya is launching his photography busin...,Places,https://www.kickstarter.com/projects/kingkoya/...,USD,111,1000,0,"Nairobi, Kenya",1760691886,1564473990,1569657990,3,0.0,Jospeph (posted by Doug),1.563865e+09
41219,Bokeh Fire: Lenses for Everyone,1465799,A simple monthly lens rental service for photo...,Camera Equipment,https://www.kickstarter.com/projects/panok/bok...,USD,21219,20000,1,"Philadelphia, PA",131591379,1415854680,1418508000,124,0.0,Pano K,1.414168e+09


In [349]:
# Change objects to numbers in dataframes for merging
df1_sample['state'] = pd.to_numeric(df1_sample['state'])
df1_sample['pledged'] = pd.to_numeric(df1_sample['pledged'])
df1_sample['goal'] = pd.to_numeric(df1_sample['goal'])
df1_sample['backers'] = pd.to_numeric(df1_sample['goal'])
df1_sample['staff_pick'] = pd.to_numeric(df1_sample['staff_pick'])
df1_sample['launched_at'] = pd.to_numeric(df1_sample['launched_at'])
df1_sample['deadline'] = pd.to_numeric(df1_sample['deadline'])


features['image'] = pd.to_numeric(features['image'])

/home/mosto/miniconda3/envs/kickstarter/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/mosto/miniconda3/envs/kickstarter/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mosto/miniconda3/envs/kickstarter/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [350]:
df1_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 3990 to 120086
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          10000 non-null  object 
 1   id            10000 non-null  object 
 2   blurb         10000 non-null  object 
 3   category      10000 non-null  object 
 4   url           10000 non-null  object 
 5   currency      10000 non-null  object 
 6   pledged       10000 non-null  float64
 7   goal          10000 non-null  float64
 8   state         10000 non-null  int64  
 9   location      9987 non-null   object 
 10  creator_id    9987 non-null   object 
 11  launched_at   9987 non-null   float64
 12  deadline      9987 non-null   float64
 13  backers       10000 non-null  float64
 14  staff_pick    9987 non-null   float64
 15  creator_name  9987 non-null   object 
 16  created_at    9987 non-null   float64
dtypes: float64(7), int64(1), object(9)
memory usage: 1.4+ MB


'https://www.kickstarter.com/projects/1718416050/a-maze-me-custom-hand-designed-name-mazes?ref=discovery_category_newest'

In [315]:
keys=df1_sample.index
state=df1_sample.state
pledged=df1_sample.pledged
goal=df1_sample.goal
backers=df1_sample.backers
staff_pick=df1_sample.staff_pick
launched_at=df1_sample.launched_at
deadline=df1_sample.deadline
category=df1_sample.category
dictionary = dict(zip(keys,state))

In [351]:
d = {'image':keys, 'state':values, 'pledged':pledged, 'goal':goal, 'backers':backers, 'staff_pick':staff_pick,
        'launched_at':launched_at, 'deadline':deadline, 'category':category}

In [352]:
df2 = pd.DataFrame(data=d)
df2.head()

,image,state,pledged,goal,backers,staff_pick,launched_at,deadline,category
3990,3990,1,541.0,500.0,500.0,0.0,1.488243e+09,1.490831e+09,DIY
124723,124723,0,50.0,5000.0,5000.0,0.0,1.477961e+09,1.480528e+09,Painting
52182,52182,1,317.0,300.0,300.0,0.0,1.559061e+09,1.560789e+09,Comic Books
43468,43468,0,111.0,1000.0,1000.0,0.0,1.564474e+09,1.569658e+09,Places
41219,41219,1,21219.0,20000.0,20000.0,0.0,1.415855e+09,1.418508e+09,Camera Equipment


In [353]:
left_merged = pd.merge(features, df2,
                        how="left", on=["image"])
left_merged.head()
left_merged.shape


(4929, 21)

In [336]:
left_merged.loc[left_merged['image']==75751]

,image,dullness,brightness,average_pixel_width,state,pledged,goal,backers,staff_pick,launched_at,deadline,category
20,75751,0.0,0.0,0.699906,1,3892.0,1000.0,1000.0,1.0,1.433860e+09,1.436452e+09,Pop


In [355]:
features_with_state=left_merged
features_with_state.head()

,image,dullness,brightness,average_pixel_width,average_color,average_red,average_green,average_blue,image_size,temp_size,...,height,blurrness,state,pledged,goal,backers,staff_pick,launched_at,deadline,category
0,94899,0.00,60.88,0.768534,"[240.86637708875867, 212.18375311957465, 138.9...",0.944574,0.832093,0.545000,39944,"(1024, 576)",...,576,166.027659,0,60.00,5000.0,5000.0,0.0,1.302576e+09,1.306464e+09,Graphic Design
1,162481,0.00,0.00,5.192227,"[118.4811774359809, 128.5906219482422, 125.515...",0.464632,0.504277,0.492219,165701,"(1024, 576)",...,576,791.188786,0,326.00,1000.0,1000.0,0.0,1.303708e+09,1.306300e+09,Electronic Music
2,201573,0.00,0.00,5.105082,"[102.67853122287326, 92.08284335666232, 180.79...",0.402661,0.361109,0.708983,140637,"(1024, 576)",...,576,196.583446,0,215.00,500.0,500.0,0.0,1.578081e+09,1.580612e+09,Crochet
3,159292,0.00,0.00,2.721759,"[75.76579604633861, 110.38401914242864, 143.18...",0.297121,0.432879,0.561527,250948,"(1552, 873)",...,873,24.676342,1,407.01,300.0,300.0,0.0,1.311861e+09,1.313108e+09,Conceptual Art
4,68557,7.93,0.00,0.367261,"[89.36553211464201, 97.02202309254733, 109.570...",0.350453,0.380479,0.429687,963059,"(1552, 873)",...,873,2.093685,1,14041.00,12000.0,12000.0,1.0,1.401988e+09,1.405099e+09,World Music


In [282]:
# Sanity Check that images match and campaign success/fail state is correct

df1_sample.loc[148087]['url']

'https://www.kickstarter.com/projects/1718416050/a-maze-me-custom-hand-designed-name-mazes?ref=discovery_category_newest'

In [285]:
# Sanity Check that images match and campaign success/fail state is correct

df1_sample.loc[90626]['url']

'https://www.kickstarter.com/projects/1544059256/choose-and-fund-wfmus-first-and-last-billboard?ref=discovery_category_newest'

In [286]:
# Sanity Check that images match and campaign success/fail state is correct

df1_sample.loc[75751]['url']

'https://www.kickstarter.com/projects/davidchen/david-chens-first-cello-ep?ref=discovery_category_newest'

In [356]:
# Serialize the data table containing the scraped HTML for each project
joblib.dump(features_with_state, '/home/mosto/Documents/insight/kickstarter-project/features_with_state_1.pkl')

['/home/mosto/Documents/insight/kickstarter-project/features_with_state_1.pkl']